In [ ]:
import requests
import sqlite3
import time

In [ ]:
#Exploring the data for one month
data_2020_01 = requests.get("https://api.nytimes.com/svc/archive/v1/2020/1.json?api-key=[YOUR API KEY]")
data_2020_01_json = data_2020_01.json()
data_2020_01_articles = data_2020_01_json['response']['docs']

In [ ]:
#Function to generate urls for each month in 2020
def generate_urls():
    month_range = range(1,13)
    api_key = "[YOUR API KEY]"
    urls = []
    for month in month_range:
        url = (f"https://api.nytimes.com/svc/archive/v1/2020/{month}.json?api-key={api_key}")
        urls.append(url)
    return urls

In [ ]:
#Create and connect to DB
connection = sqlite3.connect('nytimes_articles3.db')
cur = connection.cursor()
#Create the table
cur.execute('''CREATE TABLE articles(_id TEXT, lead_pargraph TEXT,  web_url TEXT, main_headline TEXT, pub_date TEXT, news_desk TEXT, section_name TEXT, type_of_material TEXT, word_count INT)''')


In [ ]:
#Generate URLs for each month in 2020
month_urls = generate_urls()

#Going to each month URL, getting the data and storing it in the database
#Includes sleep as suggested by the api docs

for month in month_urls:
    data = requests.get(month)
    data_json = data.json()
    data_articles = data_json['response']['docs']   
    values_for_db = []

    for article in data_articles:
        row = (article['_id'], article['lead_paragraph'], article['web_url'], article['headline']['main'], article['pub_date'], article['news_desk'], article['section_name'], article['type_of_material'], article['word_count'])
        values_for_db.append(row)
    
    cur.executemany('INSERT INTO articles VALUES (?,?,?,?,?,?,?,?,?)', values_for_db)
    time.sleep(6)
    connection.commit()

connection.close()